# How and Why BatchNorm Works - Part 2


In Part 1, we showed how using batch normalization can help train neural networks by reducing internal covariate shift and smoothing gradient updates. We also touch upon the idea that it makes the loss surface smoother and more convex (residual connections have a similar effect). 

In their 2018 paper, Santukar et al. argue that internal covariate shift isn't substantially reduced at all. In fact, when additional noise is added to the activations to force covariate shift, batch normalization continues to improve training performance. This is despite no significant reduction in covariate shift. They argue that the major reason for improved performance is the smoother loss surface.

However, we saw in our own experiments using a shallow neural network that the reduction in covariate shift is substantial. Let's get to the bottom of this.

In this experiment, we will start by reproducing the results from Santukar et al. Then we will try and understand why their results and our previous experiment differ.

## Let's start as usual by setting up the dataset
We will use the CIFAR-10 dataset for this experiment.




In [1]:
from deepkit import load_CIFAR10
train_loader, test_loader = load_CIFAR10()

In [2]:
for images, labels in train_loader:
    batch = images
    break

## Next, let's set up the model
We'll be using the standard VGG-Net architecture as per the paper.






In [20]:
import jax
import jax.numpy as jnp
import flax.nnx as nnx

class VGGBlock(nnx.Module):
    def __init__(self, in_features: int, out_features: int, rngs: nnx.Rngs):
        self.conv = nnx.Conv(in_features=in_features, 
                             out_features=out_features,
                             kernel_size=(3, 3), 
                             padding='SAME', 
                             rngs=rngs)
        self.bn = nnx.BatchNorm(num_features=out_features, use_running_average=False, 
                                rngs=rngs)

    def __call__(self, x, training: bool):
        x = self.conv(x)
        if self.bn:
            x = self.bn(x, use_running_average=not training)
        x = nnx.relu(x)
        return x

class VGGNet(nnx.Module):
    def __init__(self, rngs: nnx.Rngs):
        self.convs = [
            VGGBlock(in_features=3, out_features=64,  rngs=rngs),
            VGGBlock(in_features=64, out_features=128, rngs=rngs),
            VGGBlock(in_features=128, out_features=256, rngs=rngs),
            VGGBlock(in_features=256, out_features=256, rngs=rngs),
            VGGBlock(in_features=256, out_features=512, rngs=rngs),
            VGGBlock(in_features=512, out_features=512, rngs=rngs),
            VGGBlock(in_features=512, out_features=512, rngs=rngs),
            VGGBlock(in_features=512, out_features=512, rngs=rngs),
        ]

        self.fc1 = nnx.Linear(in_features=512, out_features=512, rngs=rngs)
        self.fc2 = nnx.Linear(in_features=512, out_features=512, rngs=rngs)
        self.out = nnx.Linear(in_features=512, out_features=10,  rngs=rngs)

    def __call__(self, x, training: bool = True):
        max_pool_after = ["block_1", "block_2", "block_4", "block_6", "block_8"]
        for layer in self.convs:
            x = layer(x, training)
            if x.name in max_pool_after:
                x = nnx.max_pool(x, window_shape=(2, 2), strides=(2, 2)),
        x = nnx.relu1(self.fc1(x))
        x = nnx.relu2(self.fc2(x))
        x = self.out(x)
        return x

In [22]:
rng_key = jax.random.key(1337)
rngs = nnx.Rngs(rng_key)
model = VGGNet(rngs=rngs)
graphdef, state = nnx.split(model)
nnx.display(state)

State({
  'convs': {
    0: {
      'bn': {
        'bias': VariableState(
          type=Param,
          value=Array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
                 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
                 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
                 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)
        ),
        'mean': VariableState(
          type=BatchStat,
          value=Array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
                 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
                 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
                 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)
        ),
        'scale': VariableState(
          type=Param,
          value=Array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,